In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import cv2
import imutils
import numpy as np
import matplotlib.pyplot as plt
import time
from os import listdir

%matplotlib inline

In [16]:
import import_ipynb
import Data_Augmentation
import Data_Preprocessing
import Data_Visualization
import model_architecture

# Data Augmentation

In [11]:
augmented_path = 'augmented_data/'

yes_path = 'data/yes'
no_path = 'data/no'

Data_Augmentation.augment_data(file_dir=yes_path, n_generated_samples=6, save_to_dir=augmented_path+'aug_yes')
Data_Augmentation.augment_data(file_dir=no_path, n_generated_samples=9, save_to_dir=augmented_path+'aug_no')

Y148.JPG
Y29.jpg
Y160.JPG
Y39.jpg
Y10.jpg
Y79.jpg
Y14.jpg
Y49.JPG
Y253.JPG
Y31.jpg
Y34.jpg
Y109.JPG
Y62.jpg
Y181.jpg
Y99.JPG
Y156.JPG
Y44.JPG
Y6.jpg
Y3.jpg
Y102.jpg
Y106.jpg
Y188.jpg
Y168.jpg
Y22.jpg
Y98.JPG
Y161.JPG
Y40.JPG
Y45.JPG
Y182.JPG
Y1.jpg
Y257.jpg
Y81.jpg
Y60.jpg
Y77.jpg
Y153.jpg
Y24.jpg
Y163.JPG
Y32.jpg
Y7.jpg
Y33.jpg
Y194.jpg
Y65.JPG
Y96.jpg
Y252.jpg
Y187.jpg
Y259.JPG
Y70.jpg
Y116.JPG
Y86.JPG
Y91.jpg
Y78.jpg
Y107.jpg
Y66.JPG
Y16.JPG
Y38.jpg
Y61.jpg
Y158.JPG
Y180.jpg
Y243.JPG
Y19.JPG
Y247.JPG
Y251.JPG
Y69.jpg
Y101.jpg
Y146.JPG
Y67.JPG
Y2.jpg
Y100.JPG
Y246.JPG
Y18.JPG
Y53.jpg
Y105.jpg
Y157.JPG
Y90.jpg
Y13.jpg
Y162.jpg
Y169.jpg
Y164.JPG
Y103.jpg
Y8.jpg
Y159.JPG
Y114.JPG
Y258.JPG
Y249.JPG
Y4.jpg
Y155.JPG
Y147.JPG
Y85.JPG
Y165.JPG
Y192.JPG
Y112.JPG
Y245.jpg
Y21.jpg
Y59.JPG
Y92.png
Y254.jpg
Y170.JPG
Y82.jpg
Y41.jpg
Y15.jpg
Y255.JPG
Y55.jpg
Y58.JPG
Y186.jpg
Y30.jpg
Y113.JPG
Y35.jpg
Y193.JPG
Y195.JPG
Y37.jpg
Y248.JPG
Y117.JPG
Y20.jpg
Y244.JPG
Y71.JPG
Y25.jpg
Y89.JPG
Y256.JPG
Y166.J

In [12]:
aug_yes_path = 'augmented_data/aug_yes'
aug_no_path = 'augmented_data/aug_no'

m_aug_yes = len(listdir(aug_yes_path))
m_aug_no = len(listdir(aug_no_path))

m_aug = m_aug_yes + m_aug_no

print(m_aug)

print("% of tumor cases     = " , m_aug_yes*100/m_aug)
print("% of non-tumor cases = " , m_aug_no*100/m_aug)

2064
% of tumor cases     =  52.56782945736434
% of non-tumor cases =  47.43217054263566


# Data Preprocessing

In [ ]:
augmented_yes = augmented_path + 'yes' 
augmented_no = augmented_path + 'no'

IMG_WIDTH, IMG_HEIGHT = (240, 240)

X, y = Data_Preprocessing.load_data([augmented_yes, augmented_no], (IMG_WIDTH, IMG_HEIGHT))

# Data Visualization

In [ ]:
Data_Visualization.plot_sample_images(X, y)

# Split the data into train, validation and test sets

In [ ]:
def split_data(X, y, test_size=0.2):
    X_train, X_test_val, y_train, y_test_val = train_test_split(X, y, test_size=test_size)
    X_test, X_val, y_test, y_val = train_test_split(X_test_val, y_test_val, test_size=0.5)
    
    return X_train, y_train, X_val, y_val, X_test, y_test

In [ ]:
X_train, y_train, X_val, y_val, X_test, y_test = split_data(X, y, test_size=0.3)

# Load the model architecture

In [ ]:
IMG_SHAPE = (IMG_WIDTH, IMG_HEIGHT, 3)

In [ ]:
model = model_architecture.build_model(IMG_SHAPE)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
log_file_name = f'brain_tumor_detection_cnn_{int(time.time())}'
tensorboard = TensorBoard(log_dir=f'logs/{log_file_name}')

In [ ]:
filepath="cnn-parameters-improvement-{epoch:02d}-{val_acc:.2f}"
checkpoint = ModelCheckpoint("models/{}.model".format(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max'))

# Training The Model

In [ ]:
start_time = time.time()

model.fit(x=X_train, y=y_train, batch_size=32, epochs=10, validation_data=(X_val, y_val), callbacks=[tensorboard, checkpoint])

end_time = time.time()
execution_time = (end_time - start_time)
print(execution_time)

# Plot the model loss and accuracy on training and validation sets

In [ ]:
history = model.history.history

In [ ]:
plot_metrics(history) 

# Test set loss and accuracy with best model

In [ ]:
best_model = load_model(filepath='models/cnn-parameters-improvement-23-0.91.model')

In [ ]:
loss, acc = best_model.evaluate(x=X_test, y=y_test)

In [ ]:
print (f"Test Loss = {loss}")
print (f"Test Accuracy = {acc}")

In [ ]:
y_test_prob = best_model.predict(X_test)
y_val_prob = best_model.predict(X_val)

In [ ]:
f1score = model_architecture.compute_f1_score(y_test, y_test_prob)
print(f"F1 score: {f1score}")

f1score_val = model_architecture.compute_f1_score(y_val, y_val_prob)
print(f"F1 score: {f1score_val}")